# MSHA Mines + Violations

The data sets are on the [MSHA data sets page](https://arlweb.msha.gov/opengovernmentdata/ogimsha.asp)

In [1]:
import pandas as pd
from IPython.display import display

Import the [mine definition file](https://arlweb.msha.gov/opengovernmentdata/DataSets/Mines_Definition_File.txt).

In [2]:
# import from local storage
#mines_definition = pd.read_table('Mines_Definition_File.txt', sep='|')

In [3]:
# import from the MSHA website
mines_definition = pd.read_table(
    'https://arlweb.msha.gov/opengovernmentdata/DataSets/Mines_Definition_File.txt', 
    sep='|')

In [4]:
mines_definition.shape

(59, 5)

In [5]:
with pd.option_context('display.max_colwidth', 9999):
     display(mines_definition.head(2))

,TABLE_NAME,COLUMN_NAME,DATA_TYPE,DATA_LENGTH,FIELD_DESCRIPTION
0,MINES,MINE_ID,VARCHAR2,7,"Identification number assigned to the mine by MSHA. It is a unique primary key to join to the Inspections, Mine Address, Accidents, Annual Employ/Prod and Qrtly Employ/Prod tables."
1,MINES,CURRENT_MINE_NAME,VARCHAR2,50,Name of the mine as designated on the Legal ID Form (LID) or Mine Information Form (MIF).


Import the [violations definition file](https://arlweb.msha.gov/opengovernmentdata/DataSets/violations_Definition_File.txt).

In [6]:
#read file from local storage
#violations_definition = pd.read_table('violations_Definition_File.txt',
#                                 sep='|',
#                                 index_col='COLUMN_NAME')

In [7]:
#read file from website
violations_definition = pd.read_table(
    'https://arlweb.msha.gov/opengovernmentdata/DataSets/violations_Definition_File.txt', 
    sep='|',
    index_col='COLUMN_NAME'
)

In [8]:
with pd.option_context('display.max_colwidth', 9999):
     display(violations_definition.head(2))

,TABLE_NAME,DATA_TYPE,DATA_LENGTH,FIELD_DESCRIPTION
COLUMN_NAME,,,,
EVENT_NO,VIOLATIONS,VARCHAR2,7,"A preassigned number used to identify each inspection, investigation, audit, survey, etc. Use event_no to join to the Inspections table which can be joined to the Mines table with mine_id."
INSPECTION_BEGIN_DT,VIOLATIONS,DATE,10,Start date of the inspection (mm/dd/yyyy).


Read the mines list file.

In [9]:
from os import access, R_OK, unlink
from datetime import datetime

import requests


def download(url, *, to, _chunk_size=8 << 10):
    '''
    Stream contents of url to file
    '''
    # Arbitrarily chose to write in 8KiB chunks. Could tweak after profiling.
    r = requests.get(url, stream=True)
    r.raise_for_status()
    try:
        with open(to, 'wb') as fp:
            for chunk in r.iter_content(chunk_size=_chunk_size):
                fp.write(chunk)
    except:
        unlink(to)


#try:
#    st = stat('Mines.zip')
#    # TODO: Which GMT
#    # Locales can mess this up, but let's assume only en_US for now.
#    headers = {'If-Modified-Since':
#                   st.mtime
#                     .astimezone()
#                     .strftime('%a, %d %b %Y %H:%M:%S GMT')}

# If you have downloaded the Mines.zip and Violations.zip files manually, the script
# will read your local files.
# Otherwise it will download them from MSHA.

if not access('Mines.zip', R_OK):
    download('https://arlweb.msha.gov/opengovernmentdata/DataSets/Mines.zip',
             to='Mines.zip')
if not access('Violations.zip', R_OK):
    download('https://arlweb.msha.gov/opengovernmentdata/DataSets/Violations.zip',
             to='Violations.zip')

In [10]:
from zipfile import ZipFile

with ZipFile('Mines.zip', 'r') as zf:
    print(zf.namelist())
    with zf.open('Mines.txt') as fp:
        mines = pd.read_table(fp, encoding='latin-1', sep='|', parse_dates=True)

# to watch the download in a terminal
# du -h Mines.zip

['Mines.txt']


In [11]:
#from zipfile import ZipFile
#
#with ZipFile('Mines.zip', 'r') as zf:
#    print(zf.namelist())
#    with zf.open('Mines.txt') as fp:
#        mines = pd.read_table(fp, encoding='latin-1', sep='|', 
#                              usecols=["MINE_ID", "CURRENT_MINE_NAME"],
#                              parse_dates=True)

In [12]:
mines.shape

(86880, 59)

In [13]:
mines.columns

Index(['MINE_ID', 'CURRENT_MINE_NAME', 'COAL_METAL_IND', 'CURRENT_MINE_TYPE',
       'CURRENT_MINE_STATUS', 'CURRENT_STATUS_DT', 'CURRENT_CONTROLLER_ID',
       'CURRENT_CONTROLLER_NAME', 'CURRENT_OPERATOR_ID',
       'CURRENT_OPERATOR_NAME', 'STATE', 'BOM_STATE_CD', 'FIPS_CNTY_CD',
       'FIPS_CNTY_NM', 'CONG_DIST_CD', 'COMPANY_TYPE',
       'CURRENT_CONTROLLER_BEGIN_DT', 'DISTRICT', 'OFFICE_CD', 'OFFICE_NAME',
       'ASSESS_CTRL_NO', 'PRIMARY_SIC_CD', 'PRIMARY_SIC', 'PRIMARY_SIC_CD_1',
       'PRIMARY_SIC_CD_SFX', 'SECONDARY_SIC_CD', 'SECONDARY_SIC',
       'SECONDARY_SIC_CD_1', 'SECONDARY_SIC_CD_SFX', 'PRIMARY_CANVASS_CD',
       'PRIMARY_CANVASS', 'SECONDARY_CANVASS_CD', 'SECONDARY_CANVASS',
       'CURRENT_103I', 'CURRENT_103I_DT', 'PORTABLE_OPERATION',
       'PORTABLE_FIPS_ST_CD', 'DAYS_PER_WEEK', 'HOURS_PER_SHIFT',
       'PROD_SHIFTS_PER_DAY', 'MAINT_SHIFTS_PER_DAY', 'NO_EMPLOYEES',
       'PART48_TRAINING', 'LONGITUDE', 'LATITUDE', 'AVG_MINE_HEIGHT',
       'MINE_GAS_CATEGO

In [14]:
#mines = pd.read_table('Mines.txt', encoding='latin-1', sep='|', 
#                       usecols=["MINE_ID", "CURRENT_MINE_NAME"])
#mines.head(5)

In [15]:
#mines.head(2)
with pd.option_context('display.max_colwidth', 9999):
     display(mines.head(2))

,MINE_ID,CURRENT_MINE_NAME,COAL_METAL_IND,CURRENT_MINE_TYPE,CURRENT_MINE_STATUS,CURRENT_STATUS_DT,CURRENT_CONTROLLER_ID,CURRENT_CONTROLLER_NAME,CURRENT_OPERATOR_ID,CURRENT_OPERATOR_NAME,...,NO_NONPRODUCING_PITS,NO_TAILING_PONDS,PILLAR_RECOVERY_USED,HIGHWALL_MINER_USED,MULTIPLE_PITS,MINERS_REP_IND,SAFETY_COMMITTEE_IND,MILES_FROM_OFFICE,DIRECTIONS_TO_MINE,NEAREST_TOWN
0,100003,O'Neal Quarry & Mill,M,Surface,Active,01/22/1979,0041044,Lhoist Group,L13586,Lhoist North America,...,NaN,0.0,N,N,N,N,N,100,Approx 7 miles south of the city of Alabaster on Hwy 31.,Calera
1,100004,Brierfield Quarry,M,Surface,Active,03/04/2003,0041044,Lhoist Group,L13586,"Lhoist North America of Alabama, LLC",...,NaN,0.0,N,N,N,N,N,35,"I-65 SOUTH EXIT US 31 TO CALERA, TURN RIGHT ON AL 25 GO THRU MONTEVALLO & WILTON, MINE SIGN WILL BE ON THE LEFT APPROX. 7 MILES, FOLLOW TO MINE OFFICE ON THE LEFT.",Brierfield


In [16]:
mines.tail(2)

,MINE_ID,CURRENT_MINE_NAME,COAL_METAL_IND,CURRENT_MINE_TYPE,CURRENT_MINE_STATUS,CURRENT_STATUS_DT,CURRENT_CONTROLLER_ID,CURRENT_CONTROLLER_NAME,CURRENT_OPERATOR_ID,CURRENT_OPERATOR_NAME,...,NO_NONPRODUCING_PITS,NO_TAILING_PONDS,PILLAR_RECOVERY_USED,HIGHWALL_MINER_USED,MULTIPLE_PITS,MINERS_REP_IND,SAFETY_COMMITTEE_IND,MILES_FROM_OFFICE,DIRECTIONS_TO_MINE,NEAREST_TOWN
86878,6900009,CROSS ISLAND ROAD,M,Surface,New Mine,10/24/2016,0125183,Hang Cheing Tan,0145154,"GPPC, INC",...,NaN,NaN,N,N,N,N,N,0,Coming from Kagman goint to Capitol Hill right...,Capital Hill
86879,6900010,United Construction Corporation,M,Surface,New Mine,11/09/2016,0133643,Xiaojun Ying,0151233,United Construction Corporation,...,NaN,NaN,N,N,N,N,N,0,"From the airport to Dandan, to San Vincente, g...",chalan laulau


In [17]:
#pd.crosstab([mines.STATE], mines.CURRENT_MINE_TYPE, margins=True)

In [18]:
#from IPython.display import display
#with pd.option_context('display.max_rows', 1000):
#    display(pd.crosstab([mines.STATE, mines.CURRENT_MINE_STATUS],
#                         mines.CURRENT_MINE_TYPE, margins=True))

In [19]:
# use this to read the data and delete the last row, if it is consistent every week
#violations = pd.read_table('Violations.txt', encoding='latin-1', sep='|', \
#                           parse_dates=True, skip_footer=1)

In [20]:
#violations = pd.read_csv('Violations.txt', encoding='latin-1', sep='|', parse_dates=True)

In [21]:
#violations = pd.read_csv('Violations.txt', encoding='latin-1', sep='|', 
#                           parse_dates=["INSPECTION_BEGIN_DT"], 
#                           usecols=["MINE_ID", "INSPECTION_BEGIN_DT", "MINE_NAME", 
#                                    "PROPOSED_PENALTY", "PART_SECTION", "EVENT_NO"])

In [22]:
%%time
with ZipFile('Violations.zip', 'r') as zf:
    with zf.open('Violations.txt') as fp:
        violations = pd.read_csv(fp, encoding='latin-1', sep='|',
                               parse_dates=["INSPECTION_BEGIN_DT"],
                               infer_datetime_format=True,
                               usecols=["MINE_ID", "INSPECTION_BEGIN_DT", "MINE_NAME",
                                        "PROPOSED_PENALTY", "PART_SECTION", "EVENT_NO"])

# to watch the download in a terminal
# du -h Violations.zip

<string>:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 19.1 s, sys: 196 ms, total: 19.2 s
Wall time: 19.3 s


In [23]:
violations.shape

(2290102, 6)

In [24]:
violations.head()

,EVENT_NO,INSPECTION_BEGIN_DT,MINE_ID,MINE_NAME,PART_SECTION,PROPOSED_PENALTY
0,716120,2000-07-25,2100249.0,NAT'L STEEL PELLET CO (MINE),56.11012,55.0
1,7184308,2000-07-07,4608801.0,Aracoma Alma Mine #1,75.321(a)(1),184.0
2,6118667,2000-04-06,1513906.0,#4,75.1101-23(c)(1),55.0
3,469450,2000-10-17,4202303.0,Rockville Pit,56.14132(a),55.0
4,505424,2000-03-03,500790.0,HENDERSON OPERATIONS,57.14107(a),55.0


In [25]:
# The INSPECTION_BEGIN_DT column is MM/DD/YYYY which is stupid to sort.
violations = violations.sort_values(by = ["INSPECTION_BEGIN_DT"])
violations.head()

,EVENT_NO,INSPECTION_BEGIN_DT,MINE_ID,MINE_NAME,PART_SECTION,PROPOSED_PENALTY
434981,712745,2000-01-02,3102043.0,Senter Sand & Gravel,56.9300(a),140.0
196326,712745,2000-01-02,3102043.0,Senter Sand & Gravel,56.14100(b),55.0
334830,4087903,2000-01-02,4603408.0,No 1 Mine,75.400,655.0
501078,712745,2000-01-02,3102043.0,Senter Sand & Gravel,56.14107(a),184.0
409570,4087903,2000-01-02,4603408.0,No 1 Mine,75.513-1,55.0


In [26]:
# drop rows before a specific date
violations = violations[(violations["INSPECTION_BEGIN_DT"] >= "2017-01-01")]
violations.head()

,EVENT_NO,INSPECTION_BEGIN_DT,MINE_ID,MINE_NAME,PART_SECTION,PROPOSED_PENALTY
2193788,6397661,2017-01-02,1519325.0,Mine No. 7,75.1910(j),116.0
2264939,6397661,2017-01-02,1519325.0,Mine No. 7,72.503(d),116.0
2157937,6397661,2017-01-02,1519325.0,Mine No. 7,75.400,116.0
2162609,6397660,2017-01-02,1519325.0,Mine No. 7,NaN,NaN
2252443,6397660,2017-01-02,1519325.0,Mine No. 7,75.1725(c),1096.0


In [27]:
violations.shape

(58583, 6)

In [28]:
# identify duplicate rows, by default keep the first in each set of duplicates
violations.duplicated().value_counts()

False    50060
True      8523
dtype: int64

In [29]:
# drop duplicate rows, by default keeping the first in each set of duplicates
#violations.drop_duplicates()

In [30]:
# select rows that contain one or more words
violations[violations["MINE_NAME"]
           .str.contains("Fairport|Windsor|Baker", na = False)].head()

,EVENT_NO,INSPECTION_BEGIN_DT,MINE_ID,MINE_NAME,PART_SECTION,PROPOSED_PENALTY
2233974,6722146,2017-01-18,3503425.0,Windsor Rock Products,56.16006,116.0
2205612,6722146,2017-01-18,3503425.0,Windsor Rock Products,56.14201(b),116.0
2234504,6722146,2017-01-18,3503425.0,Windsor Rock Products,47.41(a),116.0
2216614,6751844,2017-03-01,3301993.0,Morton Salt Fairport Mine,57.12018,116.0
2233653,6751844,2017-03-01,3301993.0,Morton Salt Fairport Mine,57.12004,126.0


In [31]:
violations.index

Int64Index([2193788, 2264939, 2157937, 2162609, 2252443, 2247341, 2247370,
            2261331, 2267133, 2280110,
            ...
            2153224, 2153972, 2263280, 2283288, 2192716, 2227367, 2239938,
            2167862, 2251056, 2216537],
           dtype='int64', length=58583)

In [32]:
#for column_index in (0,25,34,36,37,46,55,56,58):
#    column_name = violations.columns[column_index]
#    with pd.option_context('display.max_colwidth', 9999):
#        display(violations_definition.loc[column_name],
#                set(map(type,
#                        violations[column_name].unique())))

In [33]:
violations.columns

Index(['EVENT_NO', 'INSPECTION_BEGIN_DT', 'MINE_ID', 'MINE_NAME',
       'PART_SECTION', 'PROPOSED_PENALTY'],
      dtype='object')

In [34]:
with pd.option_context('display.max_colwidth', 9999):
     display(violations.head(2))

,EVENT_NO,INSPECTION_BEGIN_DT,MINE_ID,MINE_NAME,PART_SECTION,PROPOSED_PENALTY
2193788,6397661,2017-01-02,1519325.0,Mine No. 7,75.1910(j),116.0
2264939,6397661,2017-01-02,1519325.0,Mine No. 7,72.503(d),116.0


In [35]:
violations.tail(2)

,EVENT_NO,INSPECTION_BEGIN_DT,MINE_ID,MINE_NAME,PART_SECTION,PROPOSED_PENALTY
2251056,6273708,2017-07-20,3300968.0,Hopedale Mine,75.400,NaN
2216537,6484107,2017-07-20,1517216.0,Cardinal,75.370(a)(1),NaN


In [36]:
# Delete the last row
violations = violations.drop(violations.tail(1).index)

In [37]:
violations.tail(2)

,EVENT_NO,INSPECTION_BEGIN_DT,MINE_ID,MINE_NAME,PART_SECTION,PROPOSED_PENALTY
2167862,7001164,2017-07-20,1202425.0,Hilsmeyer Mine,71.404(b),NaN
2251056,6273708,2017-07-20,3300968.0,Hopedale Mine,75.400,NaN


In [38]:
# counts for column content
violation_counts = violations['EVENT_NO'].value_counts()
violation_counts.head()

6312026    235
6772920    207
6272524    204
4490714    171
6484250    168
Name: EVENT_NO, dtype: int64

In [39]:
# counts for column content
#violations['CIT_ORD_SAFE'].value_counts()

In [40]:
# counts for column content
#violations['PART_SECTION'].value_counts()

In [41]:
#pd.options.display.max_rows=1000
#pd.crosstab([violations['MINE_ID']], violations['PART_SECTION'], margins=True)

In [42]:
#violations.insert(loc=9, column='CURRENT_OPERATOR_NAME', value='')

In [43]:
#violations.head(2)

In [44]:
set(violations.columns) & set(mines.columns)

{'MINE_ID'}

In [45]:
%time df = pd.crosstab([violations.MINE_ID], violations.PART_SECTION, margins=True)

CPU times: user 864 ms, sys: 93.3 ms, total: 958 ms
Wall time: 961 ms


In [46]:
rhs = mines[['MINE_ID', 'CURRENT_OPERATOR_NAME']]
rhs.set_index('MINE_ID', inplace=True)
#df[[]].join(rhs)
df['CURRENT_OPERATOR_NAME'] = df[[]].join(rhs)['CURRENT_OPERATOR_NAME']

In [47]:
df.set_index('CURRENT_OPERATOR_NAME', append=True, inplace=True)
df.head()

,PART_SECTION,40.4,41.10,41.11,41.11(a),41.11(b),41.11(c),41.11(d),41.12,41.13,41.20,...,77.807,77.809,77.900,77.900-2,77.901(b),77.902,77.903,77.904,90.102(c),All
MINE_ID,CURRENT_OPERATOR_NAME,,,,,,,,,,,,,,,,,,,,,
100003.0,Lhoist North America,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,19
100004.0,"Lhoist North America of Alabama, LLC",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
100006.0,"Martin Marietta Materials, Inc.",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
100008.0,Cheney Lime & Cement Company,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
100010.0,Alabama Stone Company,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4


In [48]:
from sys import getsizeof
pd.DataFrame.from_records([(k, getsizeof(v) / 1024**3)
                           for k, v
                           in locals().items()],
                          columns=('name', 'size (bytes)'),
                          index='name') \
            .sort_values('size (bytes)', ascending=False)

,size (bytes)
name,
mines,1.817499e-01
df,1.029344e-01
violations,1.146256e-02
rhs,7.100870e-03
_31,2.877906e-03
_47,6.755460e-04
_,6.755460e-04
violation_counts,6.477907e-04
violations_definition,2.598017e-05


We really should `del` unused objects, such as `df`.

In [49]:
%reset -f in out

Flushing input history
Flushing output cache (19 entries)


In [50]:
del df

In [51]:
from sys import getsizeof
pd.DataFrame.from_records([(k, getsizeof(v) / 1024**3)
                           for k, v
                           in locals().items()],
                          columns=('name', 'size (GB)'),
                          index='name') \
            .sort_values('size (GB)', ascending=False)

,size (GB)
name,
mines,1.817499e-01
violations,1.146256e-02
rhs,7.100870e-03
violation_counts,6.477907e-04
violations_definition,2.598017e-05
mines_definition,2.219342e-05
ZipFile,1.363456e-06
In,4.917383e-07
_ih,4.917383e-07
